In [1]:
import networkx as nx

g = nx.DiGraph()
with open('./inputs/day16.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        for j, c in enumerate(line):
            if c in ('.', 'S', 'E'):
                g.add_node((i, j, 'N'))
                g.add_node((i, j, 'S'))
                g.add_node((i, j, 'E'))
                g.add_node((i, j, 'W'))
                g.add_edge((i, j, 'N'), (i, j, 'E'), weight=1000)
                g.add_edge((i, j, 'E'), (i, j, 'N'), weight=1000)
                g.add_edge((i, j, 'E'), (i, j, 'S'), weight=1000)
                g.add_edge((i, j, 'S'), (i, j, 'E'), weight=1000)
                g.add_edge((i, j, 'S'), (i, j, 'W'), weight=1000)
                g.add_edge((i, j, 'W'), (i, j, 'S'), weight=1000)
                g.add_edge((i, j, 'W'), (i, j, 'N'), weight=1000)
                g.add_edge((i, j, 'N'), (i, j, 'W'), weight=1000)
                

            if c == 'S':
                start = (i, j, 'E')
            elif c == 'E':
                g.add_node('End!')
                g.add_edge((i, j, 'N'), 'End!', weight=0)
                g.add_edge((i, j, 'S'), 'End!', weight=0)
                g.add_edge((i, j, 'E'), 'End!', weight=0)
                g.add_edge((i, j, 'W'), 'End!', weight=0)

for n in g.nodes:
    if n[2] == 'E' and (n[0], n[1] + 1, 'E') in g.nodes:
        g.add_edge(n, (n[0], n[1] + 1, 'E'), weight=1)
    if n[2] == 'W' and (n[0], n[1] - 1, 'W') in g.nodes:
        g.add_edge(n, (n[0], n[1] - 1, 'W'), weight=1)
    if n[2] == 'S' and (n[0] + 1, n[1], 'S') in g.nodes:
        g.add_edge(n, (n[0] + 1, n[1], 'S'), weight=1)
    if n[2] == 'N' and (n[0] - 1, n[1], 'N') in g.nodes:
        g.add_edge(n, (n[0] - 1, n[1], 'N'), weight=1)

print('Answer to Day 16, Part 1:', nx.path_weight(g, nx.shortest_path(g, start, 'End!', 'weight'), 'weight'))

path_nodes = sum((p for p in nx.all_shortest_paths(g, start, 'End!', 'weight')), [])
print('Answer to Day 16, Part 2:', len(set((n[0], n[1]) for n in path_nodes if n != 'End!')))

Answer to Day 16, Part 1: 92432
Answer to Day 16, Part 2: 458


In [2]:
with open('./inputs/day17.txt', 'r') as f:
    lines = f.readlines()

def execute_program(init_registers, program):
    registers = init_registers.copy()
    instruction_pointer = 0
    while instruction_pointer < len(program):
        literal_operand = program[instruction_pointer + 1]
        combo_operand = registers.get(
            {4: 'A', 5: 'B', 6: 'C'}.get(literal_operand),
            literal_operand
        )

        if program[instruction_pointer] == 0:
            registers['A'] = int(registers['A'] / (2 ** combo_operand))
        elif program[instruction_pointer] == 1:
            registers['B'] = int(registers['B'] ^ literal_operand)
        elif program[instruction_pointer] == 2:
            registers['B'] = combo_operand % 8
        elif program[instruction_pointer] == 3:
            if registers['A'] != 0:
                instruction_pointer = literal_operand
                continue
        elif program[instruction_pointer] == 4:
            registers['B'] = int(registers['B'] ^ registers['C'])
        elif program[instruction_pointer] == 5:
            yield combo_operand % 8
        elif program[instruction_pointer] == 6:
            registers['B'] = int(registers['A'] / (2 ** combo_operand))
        elif program[instruction_pointer] == 7:
            registers['C'] = int(registers['A'] / (2 ** combo_operand))
        
        instruction_pointer += 2

registers = {
    'A': int(lines[0].strip().split()[2]),
    'B': int(lines[1].strip().split()[2]),
    'C': int(lines[2].strip().split()[2])
}

program = list(map(int, lines[4].split()[1].split(',')))

print('Answer to Day 17, Part 1:', ','.join(map(str, execute_program(registers, program))))

def get_part2_first_output(a):
    return next(
        execute_program(
            {
                'A': a,
                'B': int(lines[1].strip().split()[2]),
                'C': int(lines[2].strip().split()[2])
            },
            program
        )
    )

def reduce(num, x):
    for _ in range(x):
        num = num // 8
    return num

part2_ans = 8 ** (len(program) - 1)
for i in range(len(program)-1, -1, -1):
    init_j = reduce(part2_ans, x=i)
    j = init_j
    while True:
        o = get_part2_first_output(j)
        if program[i] == o:
            part2_ans += (j - init_j) * (8 ** i)
            break
        else:
            j += 1

print('Answer to Day 17, Part 2:', part2_ans)

Answer to Day 17, Part 1: 7,1,2,3,2,6,7,2,5
Answer to Day 17, Part 2: 202356708354602


In [3]:
import networkx as nx

g = nx.Graph()
with open('./inputs/day18.txt', 'r') as f:
    b = [tuple(map(int, l.split(','))) for l in f.readlines()]

for i in range(71):
    for j in range(71):
        g.add_node((i, j))
        if i > 0:
            g.add_edge((i, j), (i-1, j))
        if j > 0:
            g.add_edge((i, j), (i, j-1))

for x in b[:1024]:
    g.remove_node(x)

print('Answer to Day 18, Part 1:', len(nx.shortest_path(g, (0, 0), (70, 70))) - 1)

for x in b[1024:]:
    g.remove_node(x)
    try:
        nx.shortest_path(g, (0, 0), (70, 70))
    except nx.NetworkXNoPath:
        print('Answer to Day 18, Part 2:', ','.join(map(str, x)))
        break

Answer to Day 18, Part 1: 384
Answer to Day 18, Part 2: 36,10
